In [10]:
import json 
import os
import io
import collections
import math
import random

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from PIL import Image

from pprint import pprint
from shutil import copyfile, move

import keras
from keras.optimizers import Adam
import keras.backend as K
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD

In [11]:
WIDTH = 224
HEIGHT = 224
as_grey = False
batch_size = 32
test_batch_size = 32
epochs = 500

# annotation_path_train = 'U:\PycharmProjects\AnimalBiometrics\data\\annotations\\train_annotations.json'
# annotation_path_val = 'U:\PycharmProjects\AnimalBiometrics\data\\annotations\\val_annotations.json'


In [12]:
# load labels
# train_dict = {}
# train_labels=open(annotation_path_train)
# data_train = json.load(train_labels)
# for elem in data_train['annotations']:
#     train_dict[elem['image_id']] = elem['category_id']

# train_labels.close()
# training_size = len(train_dict.values())

# val_dict = {}
# val_labels=open(annotation_path_val)
# data_val = json.load(val_labels)
# for elem in data_val['annotations']:
#     val_dict[elem['image_id']] = elem['category_id']    

# val_labels.close()
# val_size = len(val_dict.values())

# for key, value in train_dict.items():
#     copyfile(os.getcwd() + '/data/train_val/{}.jpg'.format(key), os.getcwd() + '/data/train/{}/{}.jpg'.format(value, key))
    
# for key, value in val_dict.items():
#     copyfile(os.getcwd() + '/data/train_val/{}.jpg'.format(key), os.getcwd() + '/data/val/{}/{}.jpg'.format(value, key))

In [13]:
# load images with generators

train_path = os.getcwd() + '/data/Clouded_leopard_ID/train_cleaned_cropped'
test_path = os.getcwd() + '/data/Clouded_leopard_ID/test_cleaned_cropped'

# train_path = 'D://Species//train_cropped'
# test_path = 'D://Species//test_cropped'


images = []

# augmentation configuration for training
train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.1,
#         zoom_range=0.25,
        horizontal_flip=True,
        vertical_flip=True)

# augmentation configuration for testing:
validation_datagen = ImageDataGenerator(
#         rotation_range=90,
#         width_shift_range=0.25,
#         height_shift_range=0.25,
        rescale=1./255,
#         shear_range=0.25,
#         zoom_range=0.25,
#         horizontal_flip=True,
#         vertical_flip=True
)

# train batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(HEIGHT, WIDTH),
        batch_size=batch_size,
        class_mode='categorical')

# this is a similar generator, for validation data
validation_generator = validation_datagen.flow_from_directory(
        test_path,
        target_size=(HEIGHT, WIDTH),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

unique, counts = np.unique(train_generator.classes, return_counts=True)
class_weights = dict(zip(unique, counts))

Found 391 images belonging to 29 classes.
Found 114 images belonging to 29 classes.


In [14]:
def visualize(history):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [15]:
def save_pred(pred):
    ids = [file[4:-4] for file in test_generator.filenames]
    df = pd.DataFrame(np.append(ids, pred).reshape(2, len(pred)).T)
    df.columns = ['id','animal_present']
    df.to_csv('./predictions2.csv', index=False)

In [16]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

In [17]:
from keras.utils import get_file
import keras_resnet.models

def download_imagenet(depth):
    resnet_filename = 'ResNet-{}-model.keras.h5'
    resnet_resource = 'https://github.com/fizyr/keras-models/releases/download/v0.0.1/{}'.format(resnet_filename)

    filename = resnet_filename.format(depth)
    resource = resnet_resource.format(depth)
    if depth == 50:
        checksum = '3e9f4e4f77bbe2c9bec13b53ee1c2319'
    elif depth == 101:
        checksum = '05dc86924389e5b401a9ea0348a3213c'
    elif depth == 152:
        checksum = '6ee11ef2b135592f8031058820bb9e71'

    return get_file(
        filename,
        resource,
        cache_subdir='models',
        md5_hash=checksum
    )

In [9]:
run_name='inital'
date = 'Wednesday_9_19_2018'

training_size = len(train_generator.filenames)
validation_size = len(validation_generator.filenames)
n_classes = len(np.unique(list(train_generator.class_indices.keys())))

# ResNet-50 without FC layers
# resnet_model = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet',
#                                                     input_shape=(WIDTH, HEIGHT, 3), classes=n_classes)

# # Classification block
# x = Flatten(name='flatten')(resnet_model.output)
# # x = Dense(1024, activation='relu', name='fc1')(x)
# # x = Dense(512, activation='relu', name='fc2')(x)
# x = Dense(n_classes, activation='softmax', name='predictions')(x)
# model = Model(input=resnet_model.input, output=x)


# weights = backbone.download_imagenet()
# shape = (HEIGHT, WIDTH, 3)
# x = keras.layers.Input(shape)
# resnet_model = keras_resnet.models.ResNet50(x, include_top=True, freeze_bn=False, classes=n_classes)

# model = keras.models.load_model(os.getcwd() + '/models/cloud_June 18, 2018.hdf5')
model = keras.models.load_model('D:/Species_Snapshots/Wednesday_9_19_2018_inital/weights-improvement-25-0.94.hdf5')
    
# set up tensorboard for progress and analysis 
tensorboard = TensorBoard(log_dir='D:\\Species_Graphs/{}_{}'.format(date, run_name), histogram_freq=0,
                                          batch_size=batch_size, write_graph=False, write_grads=False,
                                          write_images=True)
# save model after each epoch
root = 'D:\\Species_Snapshots/{}_{}'.format(date, run_name)
if not os.path.isdir(root):
    os.mkdir(root)
file_path = root + '/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=file_path, monitor='val_acc', verbose=1, save_best_only=False)

# reduce lr on plateau of val loss
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0)

adam = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-10, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

train_steps = math.ceil(training_size / batch_size)
val_steps = math.ceil(validation_size / batch_size)

# train_steps = 1
# val_steps = 1

history = model.fit_generator(train_generator, steps_per_epoch=train_steps, epochs=epochs,
                    validation_data=validation_generator, validation_steps=val_steps, verbose=1,
                              callbacks=[tensorboard, checkpoint, reduce_lr],
#                               class_weight=class_weights, 
                             initial_epoch=25)

# model.save(os.getcwd() + '/models/cloud_pretrained_{}.hdf5'.format(datetime.now().strftime("%B %d, %Y")))
visualize(history)

Epoch 26/500
12781/12781 [==============================] - 30291s 2s/step - loss: 0.0883 - acc: 0.9731 - val_loss: 0.2397 - val_acc: 0.9425

Epoch 00026: saving model to D:\Species_Snapshots/Wednesday_9_19_2018_inital/weights-improvement-26-0.94.hdf5
Epoch 27/500
12781/12781 [==============================] - 34586s 3s/step - loss: 0.0826 - acc: 0.9750 - val_loss: 0.2349 - val_acc: 0.9432

Epoch 00027: saving model to D:\Species_Snapshots/Wednesday_9_19_2018_inital/weights-improvement-27-0.94.hdf5
Epoch 28/500
  119/12781 [..............................] - ETA: 6:40:04 - loss: 0.0765 - acc: 0.9764

KeyboardInterrupt: 

In [29]:
run_name='Individual_freeze'
date = 'Tuesday_10_2_2018'

training_size = len(train_generator.filenames) * 10
validation_size = len(validation_generator.filenames) * 10
n_classes = len(np.unique(list(train_generator.class_indices.keys())))

In [33]:
# load old model
old_model = keras.models.load_model('D:/Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-185-0.58.hdf5')
# old_model.layers.pop()

# freeze resnet
for i in range(len(old_model.layers)):
    old_model.layers[i].trainable = True

# add new class count Dense layer
# x = Dense(n_classes, activation='softmax', name='predictions')(old_model.layers[-1].output)
# model = Model(input=old_model.input, output=x)

optimizer = Adam(lr=1e-6, beta_1=0.9, beta_2=0.999, epsilon=1e-10, decay=0.0, amsgrad=False)
# optimizer = RMSprop(lr=1e-6, rho=0.9, epsilon=1e-10, decay=0.0)
old_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

# set up tensorboard for progress and analysis 
tensorboard = TensorBoard(log_dir='D:\\Species_Graphs/{}_{}'.format(date, run_name), histogram_freq=0,
                                          batch_size=batch_size, write_graph=False, write_grads=False,
                                          write_images=True)
# save model after each epoch
root = 'D:\\Species_Snapshots/{}_{}'.format(date, run_name)
if not os.path.isdir(root):
    os.mkdir(root)
file_path = root + '/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath=file_path, monitor='val_acc', verbose=1, save_best_only=False)

# reduce lr on plateau of val loss
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0)

train_steps = math.ceil(training_size / batch_size)
val_steps = math.ceil(validation_size / batch_size)

# train_steps = 1
# val_steps = 1

history = old_model.fit_generator(train_generator, steps_per_epoch=train_steps, epochs=epochs,
                    validation_data=validation_generator, validation_steps=val_steps, verbose=1,
                              callbacks=[tensorboard, checkpoint, reduce_lr],
#                               class_weight=class_weights, 
                             initial_epoch=185
                             )

# model.save(os.getcwd() + '/models/cloud_pretrained_{}.hdf5'.format(datetime.now().strftime("%B %d, %Y")))
visualize(history)

Epoch 186/500
13/13 [==============================] - 25s 2s/step - loss: 0.8557 - acc: 0.8906 - val_loss: 0.9181 - val_acc: 0.8246

Epoch 00186: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-186-0.82.hdf5
Epoch 187/500
13/13 [==============================] - 10s 757ms/step - loss: 0.8235 - acc: 0.8826 - val_loss: 0.8967 - val_acc: 0.8246

Epoch 00187: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-187-0.82.hdf5
Epoch 188/500
13/13 [==============================] - 20s 2s/step - loss: 0.7382 - acc: 0.9107 - val_loss: 0.8765 - val_acc: 0.8158

Epoch 00188: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-188-0.82.hdf5
Epoch 189/500
13/13 [==============================] - 21s 2s/step - loss: 0.6891 - acc: 0.9220 - val_loss: 0.8525 - val_acc: 0.8158

Epoch 00189: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improve

13/13 [==============================] - 19s 1s/step - loss: 0.2528 - acc: 0.9686 - val_loss: 0.4366 - val_acc: 0.9035

Epoch 00218: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-218-0.90.hdf5
Epoch 219/500
13/13 [==============================] - 17s 1s/step - loss: 0.2605 - acc: 0.9510 - val_loss: 0.4289 - val_acc: 0.9035

Epoch 00219: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-219-0.90.hdf5
Epoch 220/500
13/13 [==============================] - 21s 2s/step - loss: 0.2791 - acc: 0.9534 - val_loss: 0.4220 - val_acc: 0.9035

Epoch 00220: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-220-0.90.hdf5
Epoch 221/500
13/13 [==============================] - 18s 1s/step - loss: 0.2163 - acc: 0.9734 - val_loss: 0.4178 - val_acc: 0.9035

Epoch 00221: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-221-0.90.hdf

13/13 [==============================] - 21s 2s/step - loss: 0.1289 - acc: 0.9903 - val_loss: 0.2865 - val_acc: 0.9298

Epoch 00250: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-250-0.93.hdf5
Epoch 251/500
13/13 [==============================] - 20s 2s/step - loss: 0.1157 - acc: 0.9879 - val_loss: 0.2839 - val_acc: 0.9298

Epoch 00251: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-251-0.93.hdf5
Epoch 252/500
13/13 [==============================] - 19s 1s/step - loss: 0.1213 - acc: 0.9807 - val_loss: 0.2817 - val_acc: 0.9298

Epoch 00252: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-252-0.93.hdf5
Epoch 253/500
13/13 [==============================] - 20s 2s/step - loss: 0.1187 - acc: 0.9775 - val_loss: 0.2802 - val_acc: 0.9211

Epoch 00253: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-253-0.92.hdf

13/13 [==============================] - 20s 2s/step - loss: 0.0819 - acc: 0.9903 - val_loss: 0.2200 - val_acc: 0.9386

Epoch 00282: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-282-0.94.hdf5
Epoch 283/500
13/13 [==============================] - 21s 2s/step - loss: 0.0878 - acc: 0.9879 - val_loss: 0.2171 - val_acc: 0.9386

Epoch 00283: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-283-0.94.hdf5
Epoch 284/500
13/13 [==============================] - 22s 2s/step - loss: 0.0826 - acc: 0.9976 - val_loss: 0.2134 - val_acc: 0.9386

Epoch 00284: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-284-0.94.hdf5
Epoch 285/500
13/13 [==============================] - 21s 2s/step - loss: 0.0718 - acc: 0.9928 - val_loss: 0.2123 - val_acc: 0.9386

Epoch 00285: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-285-0.94.hdf

13/13 [==============================] - 21s 2s/step - loss: 0.0430 - acc: 1.0000 - val_loss: 0.1804 - val_acc: 0.9474

Epoch 00314: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-314-0.95.hdf5
Epoch 315/500
13/13 [==============================] - 19s 1s/step - loss: 0.0597 - acc: 0.9976 - val_loss: 0.1795 - val_acc: 0.9474

Epoch 00315: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-315-0.95.hdf5
Epoch 316/500
13/13 [==============================] - 20s 2s/step - loss: 0.0413 - acc: 0.9976 - val_loss: 0.1779 - val_acc: 0.9474

Epoch 00316: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-316-0.95.hdf5
Epoch 317/500
13/13 [==============================] - 20s 2s/step - loss: 0.0441 - acc: 0.9976 - val_loss: 0.1774 - val_acc: 0.9561

Epoch 00317: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-317-0.96.hdf

13/13 [==============================] - 21s 2s/step - loss: 0.0292 - acc: 1.0000 - val_loss: 0.1560 - val_acc: 0.9561

Epoch 00346: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-346-0.96.hdf5
Epoch 347/500
13/13 [==============================] - 20s 2s/step - loss: 0.0340 - acc: 1.0000 - val_loss: 0.1552 - val_acc: 0.9561

Epoch 00347: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-347-0.96.hdf5
Epoch 348/500
13/13 [==============================] - 23s 2s/step - loss: 0.0302 - acc: 1.0000 - val_loss: 0.1549 - val_acc: 0.9561

Epoch 00348: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-348-0.96.hdf5
Epoch 349/500
13/13 [==============================] - 21s 2s/step - loss: 0.0318 - acc: 1.0000 - val_loss: 0.1552 - val_acc: 0.9561

Epoch 00349: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-349-0.96.hdf

13/13 [==============================] - 22s 2s/step - loss: 0.0278 - acc: 1.0000 - val_loss: 0.1569 - val_acc: 0.9561

Epoch 00378: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-378-0.96.hdf5
Epoch 379/500
13/13 [==============================] - 21s 2s/step - loss: 0.0287 - acc: 0.9976 - val_loss: 0.1572 - val_acc: 0.9561

Epoch 00379: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-379-0.96.hdf5
Epoch 380/500
13/13 [==============================] - 21s 2s/step - loss: 0.0401 - acc: 0.9976 - val_loss: 0.1570 - val_acc: 0.9561

Epoch 00380: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-380-0.96.hdf5
Epoch 381/500
13/13 [==============================] - 20s 2s/step - loss: 0.0311 - acc: 1.0000 - val_loss: 0.1565 - val_acc: 0.9561

Epoch 00381: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-381-0.96.hdf

13/13 [==============================] - 22s 2s/step - loss: 0.0281 - acc: 1.0000 - val_loss: 0.1553 - val_acc: 0.9561

Epoch 00410: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-410-0.96.hdf5
Epoch 411/500
13/13 [==============================] - 18s 1s/step - loss: 0.0356 - acc: 1.0000 - val_loss: 0.1547 - val_acc: 0.9561

Epoch 00411: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-411-0.96.hdf5
Epoch 412/500
13/13 [==============================] - 18s 1s/step - loss: 0.0323 - acc: 0.9976 - val_loss: 0.1552 - val_acc: 0.9561

Epoch 00412: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-412-0.96.hdf5
Epoch 413/500
13/13 [==============================] - 22s 2s/step - loss: 0.0321 - acc: 0.9976 - val_loss: 0.1556 - val_acc: 0.9561

Epoch 00413: saving model to D:\Species_Snapshots/Tuesday_10_2_2018_Individual_freeze/weights-improvement-413-0.96.hdf

KeyboardInterrupt: 

In [ ]:
# # compute with exiting model which had not 29 classes

# # input_tensor = Input(shape=(WIDTH, HEIGHT, 3))

# # # VGG without FC layers
# # # model = keras.applications.vgg16.VGG16(include_top=False, weights="imagenet")
# # model = keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet',
# #                                      input_tensor=input_tensor, classes=2)
# # x = model(input_tensor)

# training_size = len(train_generator.filenames) * 10
# validation_size = len(validation_generator.filenames) * 10

# model = keras.models.load_model(os.getcwd() + '/models/pretrained.h5')
# model.layers.pop()
# input_layer = model.input
# resnet = model.layers[1]
# x = resnet(input_layer)
# # model.add(Dense(29, activation='softmax', name='predictions'))
# # input_tensor = model.input
# # x = model.layers[1:-2]

# for layer in model.layers[2:-1]:
#     x = layer(x)

# # Classification block
# # x = Flatten(name='flatten')(x)
# x = Dense(1024, activation='relu', name='fc1')(x)
# x = Dense(1024, activation='relu', name='fc2')(x)
# x = Dense(29, activation='softmax', name='predictions')(x)
# model = Model(input=input_layer, output=x)
# print(model.layers)
    
# # set up tensorboard for progress and analysis 
# # tensorboard = TensorBoard(log_dir=os.getcwd()+"/logs/cloud/{}".format(datetime.now().strftime("%B %d, %Y")), histogram_freq=0,
# #                                           batch_size=batch_size, write_graph=True, write_grads=True,
# #                                           write_images=True)

# # checkpoint = ModelCheckpoint(filepath='/model_checkpoints/large_dense_fine/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5', 
# #                              monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

# history = model.fit_generator(train_generator, steps_per_epoch=training_size / batch_size, epochs=epochs,
#                     validation_data=validation_generator, validation_steps=validation_size / batch_size, verbose=1,
# #                               callbacks=[tensorboard, checkpoint]
#                              )

# model.save(os.getcwd() + '/models/cloud_{}.hdf5'.format(datetime.now().strftime("%B %d, %Y")))
# visualize(history)

In [ ]:
pred = model.predict_generator(test_generator, len([name for name in os.listdir(os.getcwd() + '/data/test/all')]) // test_batch_size, verbose=1)
save_pred(pred.argmax(axis=-1))

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7649718654335443166
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9221160305
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13525547311064891257
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:17:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 9221160305
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2141044523509296848
physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:65:00.0, compute capability: 6.1"
]
